In [1]:
import pandas as pd
from lifelines import NelsonAalenFitter  # 引入NelsonAalenFitter

df = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

# 删除唯一标识符和缺失值过多的列
columns_to_drop = [
    'cyto_score_detail', 
    'renal_issue',
    'hla_high_res_10',
    'hla_low_res_6',
    'rheum_issue',
    'hla_low_res_8',
    'hla_match_drb1_low',
    'hla_match_c_low',  # Importance: 0.06591817809956657
    'rituximab',        # Importance: 0.05381579652762678
    'hla_low_res_10',   # Importance: 0.030017220216923903
    'hla_high_res_8',   # Importance: 0.02714947179965365
]
columns_to_drop1 = [
    'cyto_score_detail', 
    'renal_issue',
    'hla_high_res_10',
    'hla_low_res_6',
    'rheum_issue',
    'hla_low_res_8',
    'hla_match_drb1_low',
    'hla_match_c_low',  # Importance: 0.06591817809956657
    'rituximab',        # Importance: 0.05381579652762678
    'hla_low_res_10',   # Importance: 0.030017220216923903
    'hla_high_res_8',   # Importance: 0.02714947179965365
]

# 保留原始的 ID 列，最后输出用
id_column = test_data['ID']

df.drop(columns=columns_to_drop1, inplace=True)
test_data.drop(columns=columns_to_drop, inplace=True)


df.head()

,ID,dri_score,psych_disturb,cyto_score,diabetes,hla_match_c_high,tbi_status,arrhythmia,graft_type,vent_hist,...,karnofsky_score,hepatic_mild,tce_div_match,donor_related,melphalan_dose,cardiac,hla_match_drb1_high,pulm_moderate,efs,efs_time
0,0,N/A - non-malignant indication,No,NaN,No,NaN,No TBI,No,Bone marrow,No,...,90.0,No,NaN,Unrelated,"N/A, Mel not given",No,2.0,No,0.0,42.356
1,1,Intermediate,No,Intermediate,No,2.0,"TBI +- Other, >cGy",No,Peripheral blood,No,...,90.0,No,Permissive mismatched,Related,"N/A, Mel not given",No,2.0,Yes,1.0,4.672
2,2,N/A - non-malignant indication,No,NaN,No,2.0,No TBI,No,Bone marrow,No,...,90.0,No,Permissive mismatched,Related,"N/A, Mel not given",No,2.0,No,0.0,19.793
3,3,High,No,Intermediate,No,2.0,No TBI,No,Bone marrow,No,...,90.0,Yes,Permissive mismatched,Unrelated,"N/A, Mel not given",No,2.0,No,0.0,102.349
4,4,High,No,NaN,No,2.0,No TBI,No,Peripheral blood,No,...,90.0,No,Permissive mismatched,Related,MEL,No,2.0,No,0.0,16.223


In [2]:
import numpy as np
from scipy.stats import shapiro
import pandas as pd

# 假设df和test_data已经是预处理过的数据框

# 正态分布填补数值型数据缺失值
for col in df.columns:
    if df[col].dtype in ['int64', 'float64']:  # 只对数值型数据处理
        # 检查是否是正态分布（假设df[col]有非缺失值）
        try:
            stat, p = shapiro(df[col].dropna())  # 进行正态性检验
        except ValueError:  # 如果该列全是缺失值，跳过该列
            continue
        
        if p > 0.05:  # 认为数据服从正态分布
            mean = df[col].mean()
            std = df[col].std()
            
            # 找到缺失的位置
            missing = df[col].isnull()
            
            # 生成填补的数值数量等于缺失的数量
            fill_values = np.random.normal(mean, std, size=missing.sum())
            
            # 替换缺失值
            df.loc[missing, col] = fill_values


# 对于 object 类型的列使用基于频率的正态分布填充缺失值
for col in df.select_dtypes(include=['object']).columns:
    # 计算类别的频率（出现的概率）
    freq = df[col].value_counts(normalize=True)
    
    # 获取缺失值的位置
    missing = df[col].isnull()
    
    # 从已知的类别中，根据频率随机选择填充值
    fill_values = np.random.choice(freq.index, size=missing.sum(), p=freq.values)
    
    # 填充缺失值
    df.loc[missing, col] = fill_values

# 对 test_data 也进行相同处理
for col in test_data.select_dtypes(include=['object']).columns:
    freq = test_data[col].value_counts(normalize=True)
    missing = test_data[col].isnull()
    fill_values = np.random.choice(freq.index, size=missing.sum(), p=freq.values)
    test_data.loc[missing, col] = fill_values


for col in test_data.columns:
    if test_data[col].dtype in ['int64', 'float64']:  # 只对数值型数据处理
        try:
            stat, p = shapiro(test_data[col].dropna())  # 进行正态性检验
        except ValueError:  # 如果该列全是缺失值，跳过该列
            continue
        
        if p > 0.05:  # 认为数据服从正态分布
            mean = test_data[col].mean()
            std = test_data[col].std()
            
            # 找到缺失的位置
            missing = test_data[col].isnull()
            
            # 生成填补的数值数量等于缺失的数量
            fill_values = np.random.normal(mean, std, size=missing.sum())
            
            # 替换缺失值
            test_data.loc[missing, col] = fill_values


C:\Users\15951\anaconda3\Lib\site-packages\scipy\stats\_axis_nan_policy.py:531: UserWarning: scipy.stats.shapiro: For N > 5000, computed p-value may not be accurate. Current N is 28800.
  res = hypotest_fun_out(*samples, **kwds)
C:\Users\15951\anaconda3\Lib\site-packages\scipy\stats\_axis_nan_policy.py:531: UserWarning: scipy.stats.shapiro: For N > 5000, computed p-value may not be accurate. Current N is 24180.
  res = hypotest_fun_out(*samples, **kwds)
C:\Users\15951\anaconda3\Lib\site-packages\scipy\stats\_axis_nan_policy.py:531: UserWarning: scipy.stats.shapiro: For N > 5000, computed p-value may not be accurate. Current N is 23516.
  res = hypotest_fun_out(*samples, **kwds)
C:\Users\15951\anaconda3\Lib\site-packages\scipy\stats\_axis_nan_policy.py:531: UserWarning: scipy.stats.shapiro: For N > 5000, computed p-value may not be accurate. Current N is 23601.
  res = hypotest_fun_out(*samples, **kwds)
C:\Users\15951\anaconda3\Lib\site-packages\scipy\stats\_axis_nan_policy.py:531: User

## 认为数据属于正态分布的做法并没有让数据变好很多，以至于在部分情况下可能让数据预处理变差（这点我深感疑惑）

In [3]:
import pandas as pd
import numpy as np
from catboost import CatBoostClassifier
from sklearn.model_selection import StratifiedKFold
from lifelines.utils import concordance_index
from sklearn.metrics import accuracy_score

class ParticipantVisibleError(Exception):
    pass

def score(solution: pd.DataFrame, submission: pd.DataFrame, row_id_column_name: str) -> float:

    del solution[row_id_column_name]
    del submission[row_id_column_name]
    
    for col in submission.columns:
        if not pd.api.types.is_numeric_dtype(submission[col]):
            raise ParticipantVisibleError(f'Submission column {col} must be numeric')
    
    # 合并数据并分组计算
    merged_df = pd.concat([solution, submission], axis=1)
    merged_df_race_groups = merged_df.groupby('race_group')
    
    metric_list = []
    for _, group_df in merged_df_race_groups:
        c_index = concordance_index(
            group_df['efs_time'],
            -group_df['prediction'],  # 假设预测值越大代表风险越高
            group_df['efs']
        )
        metric_list.append(c_index)
    
    return float(np.mean(metric_list) - np.sqrt(np.var(metric_list)))

solution_df = df[['ID', 'efs', 'efs_time', 'race_group']].copy()
X = df.drop(['efs', 'efs_time', 'ID', 'race_group'], axis=1)  # 移除目标列和分组列
y = df['efs']  # 事件发生标签

# 单调性约束
monotone_constraints = {
    'hla_high_res_6': -1,
    'hla_match_a_high': -1,
    'hla_match_dqb1_low': -1,
    'hla_nmdp_6': -1,
}
monotonic_dict = {col: monotone_constraints.get(col, 0) for col in X.columns}

# 交叉验证
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
metrics = []

# 存储指标的列表
c_index_metrics = []
accuracies = []

for train_idx, test_idx in kf.split(X, y):
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
    solution_test = solution_df.iloc[test_idx].copy().reset_index(drop=True)
    
    # CatBoost 模型，其中部分参数似乎只适用于r语言，此处是可用于python
    model = CatBoostClassifier(
        learning_rate=0.03,
        depth=3,
        iterations=1000,
        l2_leaf_reg=0.5,
        verbose=500,
        task_type='CPU',
        cat_features=list(X_train.select_dtypes(include='object').columns),
        monotone_constraints=monotonic_dict
    )
    model.fit(X_train, y_train)
    
    
    pred_proba = model.predict_proba(X_test)[:, 1]  # 用于C-index
    y_pred = model.predict(X_test)                 # 用于准确率
    
    # 计算C-index，我不确定是否完全符合题目公式
    submission_test = pd.DataFrame({
        'ID': solution_test['ID'],
        'prediction': pred_proba
    })
    fold_c_index = score(solution_test, submission_test, 'ID')
    c_index_metrics.append(fold_c_index)

    # 计算准确率
    fold_accuracy = accuracy_score(y_test, y_pred)
    accuracies.append(fold_accuracy)
    
    
    print(f'Fold {len(c_index_metrics)}:')
    print(f'  C-index: {fold_c_index:.4f}')
    print(f'  Accuracy: {fold_accuracy:.4f}')
    print('-'*40)


print('\n汇总结果:')
print(f'平均 C-index: {np.mean(c_index_metrics):.4f} (±{np.std(c_index_metrics):.4f})')
print(f'调整后 C-index: {np.mean(c_index_metrics) - np.sqrt(np.var(c_index_metrics)):.4f}')
print(f'\n平均准确率: {np.mean(accuracies):.4f} (±{np.std(accuracies):.4f})')

0:	learn: 0.6903885	total: 195ms	remaining: 3m 14s
500:	learn: 0.5884456	total: 14.1s	remaining: 14s
999:	learn: 0.5792066	total: 29s	remaining: 0us
Fold 1:
  C-index: 0.6211
  Accuracy: 0.6698
----------------------------------------
0:	learn: 0.6907329	total: 32.3ms	remaining: 32.3s
500:	learn: 0.5940500	total: 16s	remaining: 15.9s
999:	learn: 0.5840393	total: 32.6s	remaining: 0us
Fold 2:
  C-index: 0.6428
  Accuracy: 0.6816
----------------------------------------
0:	learn: 0.6907765	total: 39.3ms	remaining: 39.3s
500:	learn: 0.5945591	total: 16.4s	remaining: 16.3s
999:	learn: 0.5856975	total: 32.8s	remaining: 0us
Fold 3:
  C-index: 0.6417
  Accuracy: 0.6875
----------------------------------------
0:	learn: 0.6906322	total: 39.8ms	remaining: 39.8s
500:	learn: 0.5922731	total: 17.1s	remaining: 17s
999:	learn: 0.5829832	total: 35.6s	remaining: 0us
Fold 4:
  C-index: 0.6305
  Accuracy: 0.6849
----------------------------------------
0:	learn: 0.6906037	total: 57.6ms	remaining: 57.5s
5

In [4]:
test_data = test_data[X.columns]

test_predictions = model.predict_proba(test_data)[:, 1]  # 获取属于类别1的概率

# 输出预测的前几个结果
print("测试集前几个预测的概率:", test_predictions[:10])



测试集前几个预测的概率: [0.17383564 0.64654589 0.09964097]


In [5]:
submission = pd.DataFrame({'ID': id_column, 'prediction': test_predictions})
submission.to_csv('submission.csv', index=False)
print("提交结果已保存为 'submission.csv'")

提交结果已保存为 'submission.csv'
